In [1]:
import os
import tensorflow
import keras
import json

In [ ]:
path = 'C:/users/trifo/desktop/MUMT 203 AMT/package_aa'
tie_list = []
# count = 0
for idx, dirname in enumerate(os.listdir(path)):
    # print(dirname)
    for filename in os.listdir(f'{path}/{dirname}'):
        if filename.endswith(".semantic"):
            f = open(f'{path}/{dirname}/{filename}', 'r')
            # read = f.read().split()
            total = f.read()
            if "tie" in total:
                tie_list.append(filename.split('.')[0])
            # print(read)
            # for j in read:
            #     if "keySignature" in j:
            #         y_sequences.add(j)
            # y_sequences.append(f.read().split())
# print(y_sequences)
print(tie_list)

In [30]:
tie_list[0]

'000051661-1_1_2'

In [27]:
json_path = 'C:/users/trifo/desktop/AMT/test_order.json'
f = open(json_path)
order = json.load(f)

In [32]:
list(order)[0]

'000051650-1_1_1.wav'

In [36]:
import numpy as np

In [37]:
example = np.load('C:/Users/trifo/Desktop/AMT/batches/y/batch_1/000051653-1_2_1.npy')

In [38]:
example

array([ 959,  613,  166,  964,  350,  350,  975,  745,  169,  403,  832,
       1031, 1031,  975], dtype=int64)

In [33]:
for i in tie_list:
    order.pop(i + '.wav', None)

In [35]:
with open('C:/users/trifo/desktop/AMT/no_tie.json', 'w') as f:
    json.dump(order, f)

In [3]:
no_tie = json.load(open('C:/users/trifo/desktop/AMT/no_tie.json', 'r'))

In [6]:
f = open('C:/users/trifo/desktop/AMT/no_tie.json')
order = json.load(f)
mapping = open('C:/users/trifo/desktop/AMT/w2i_aav2.json')
w2i = json.load(mapping)
sorted_order = list(sorted(order.items(), key=lambda item: item[1], reverse=True))
max_len = 0
# for img in sorted_order:
#     name = img[0].split('.')[0]
#     f_in = open(f'C:/users/trifo/desktop/MUMT 203 AMT/package_aa/{name}/{name}.semantic', 'r')
#     notes = [w2i[note] for note in f_in.read().split() if not ('barline' in note or 'keySignature' in note or 'clef' in note or 'timeSignature' in note)]
#     if len(notes) > max_len:
#         max_len = len(notes)
# max_len

In [2]:
import os, sys
import fnmatch
import cv2
import numpy as np
import string
import time
import json
import math

from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Dense, LSTM, Reshape, BatchNormalization, Input, Conv2D, MaxPool2D, Lambda, Bidirectional
from tensorflow.keras.models import Model
from tensorflow.keras.activations import relu, sigmoid, softmax
import tensorflow.keras.backend as K
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras import layers

In [3]:
class GlobalMaxPoolAcrossTime(layers.Layer):
    def __init__(self, **kwargs):
        super(GlobalMaxPoolAcrossTime, self).__init__(**kwargs)

    # (Batch_size, time, y, x, channels) -> (Batch_size, 1, y, x, channels)
    def call(self, inputs):
        shape = inputs.shape[2]
        if not inputs.shape[2]:
            shape = 64
        pixels = math.floor(shape/64)
        # pixels = int(shape/64)
        # print('pixels')
        # print('pixels',pixels)
        pool_list = []
        for i in range(0, pixels*64, pixels):
            pool_list.append(K.max(inputs[:,:,i:i+pixels], axis=2, keepdims=True))
        # return tensorflow.keras.backend.max(inputs, axis=1, keepdims=True)
        # print(len(pool_list))
        return layers.Concatenate(axis=2)(pool_list)

In [7]:
# input with shape of height=32 and width=128 
inputs = Input(shape=(128,None,1))
# print(inputs.shape[0]/2)
 
# convolution layer with kernel size (3,3)
conv_1 = Conv2D(64, (3,3), activation = 'relu', padding='same')(inputs)
print(conv_1)
# poolig layer with kernel size (2,2)
pool_1 = MaxPool2D(pool_size=(2, 1))(conv_1)
print(pool_1)
 
conv_2 = Conv2D(128, (3,3), activation = 'relu', padding='same')(pool_1)
print(conv_2)

pool_2 = MaxPool2D(pool_size=(2, 1))(conv_2)
print(pool_2)
 
conv_3 = Conv2D(256, (3,3), activation = 'relu', padding='same')(pool_2)
print(conv_3)
 
conv_4 = Conv2D(256, (3,3), activation = 'relu', padding='same')(conv_3)
print(conv_4)
# poolig layer with kernel size (2,1)
pool_4 = MaxPool2D(pool_size=(4, 1))(conv_4)
print(pool_4)

conv_5 = Conv2D(512, (3,3), activation = 'relu', padding='same')(pool_4)
print(conv_5)
# Batch normalization layer
batch_norm_5 = BatchNormalization()(conv_5)
print(batch_norm_5)
 
conv_6 = Conv2D(512, (3,3), activation = 'relu', padding='same')(batch_norm_5)
print(conv_6)
batch_norm_6 = BatchNormalization()(conv_6)
print(batch_norm_6)
pool_6 = MaxPool2D(pool_size=(4, 1))(batch_norm_6)
print(f'pool_6: {pool_6}')

global_pool = GlobalMaxPoolAcrossTime()(pool_6)
print(f'global_pool: {global_pool}')

conv_7 = Conv2D(512, (2,2), activation = 'relu')(global_pool)
print(f'conv_7: {conv_7}')

squeezed = Lambda(lambda x: K.squeeze(x, 1))(conv_7)
print(squeezed)
 
# bidirectional LSTM layers with units=128
blstm_1 = Bidirectional(LSTM(128, return_sequences=True, dropout = 0.2))(squeezed)
blstm_2 = Bidirectional(LSTM(128, return_sequences=True, dropout = 0.2))(blstm_1)

outputs = Dense(len(w2i)+1, activation = 'softmax')(blstm_2)

# model to be used at test time
act_model = Model(inputs, outputs)

KerasTensor(type_spec=TensorSpec(shape=(None, 128, None, 64), dtype=tf.float32, name=None), name='conv2d_7/Relu:0', description="created by layer 'conv2d_7'")
KerasTensor(type_spec=TensorSpec(shape=(None, 64, None, 64), dtype=tf.float32, name=None), name='max_pooling2d_4/MaxPool:0', description="created by layer 'max_pooling2d_4'")
KerasTensor(type_spec=TensorSpec(shape=(None, 64, None, 128), dtype=tf.float32, name=None), name='conv2d_8/Relu:0', description="created by layer 'conv2d_8'")
KerasTensor(type_spec=TensorSpec(shape=(None, 32, None, 128), dtype=tf.float32, name=None), name='max_pooling2d_5/MaxPool:0', description="created by layer 'max_pooling2d_5'")
KerasTensor(type_spec=TensorSpec(shape=(None, 32, None, 256), dtype=tf.float32, name=None), name='conv2d_9/Relu:0', description="created by layer 'conv2d_9'")
KerasTensor(type_spec=TensorSpec(shape=(None, 32, None, 256), dtype=tf.float32, name=None), name='conv2d_10/Relu:0', description="created by layer 'conv2d_10'")
KerasTensor

In [8]:
def val_gen():
    path_x = 'D:/AMT/batches/aa_piano/validate_x'
    path_y = 'D:/AMT/batches/aa_piano/validate_y'
    for idx, dirname in enumerate(os.listdir(path_x)):
        validate_img = []
        validate_txt = []
        validate_input_length = []
        validate_label_length = []
        # max_label_len = 0
        for filename in os.listdir(f'{path_x}/{dirname}'):
            # Images
            img = cv2.imread(f'{path_x}/{dirname}/{filename}', cv2.IMREAD_GRAYSCALE)
            # img = img.reshape(img.shape[1], img.shape[0])
            # (W, H) --> (W, H, 1)
            img = np.expand_dims(img, axis=2)
            # Normalize image
            img = img / 255.
            validate_img.append(img)

            # Text Targets
            text = np.load(f'{path_y}/{dirname}/{filename}'.split('.')[0] + '.npy')
            # if len(text) > max_label_len:
            #     max_label_len = len(text)
            validate_txt.append(text)
            validate_label_length.append(len(text))
            validate_input_length.append(63)
        validate_padded_txt = pad_sequences(validate_txt, padding='post', maxlen=50)

        yield np.array(validate_img), np.array(validate_padded_txt), np.array(validate_input_length), np.array(validate_label_length)

In [9]:
val = val_gen()
valid_img, valid_padded_txt, valid_input_length, valid_label_length = next(val)

In [10]:
# load the saved best model weights
act_model.load_weights('best_model.hdf5')
# act_model.load_weights('best_model.hdf5')
 
# predict outputs on validation images
# img = cv2.imread('D:/AMT/batches/aa_piano/validate_x/batch_10/000125088-24_1_1.png', cv2.IMREAD_GRAYSCALE)
# img = np.expand_dims(img, axis=2)
# img = np.expand_dims(img, axis=0)
# print(img.shape)
# # Normalize image
# img = img / 255.
prediction = act_model.predict(valid_img[:5])

# prediction = act_model.predict(img)

# use CTC decoder
out = K.get_value(K.ctc_decode(prediction, input_length=np.ones(prediction.shape[0])*prediction.shape[1],
                         greedy=False)[0][0])

i2w = json.load(open('C:/Users/trifo/Desktop/AMT/i2w_aav2.json'))

1/1 [==============================] - 7s 7s/step


In [11]:
out

array([[820,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,
         -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,
         -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,
         -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,
         -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1],
       [820,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,
         -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,
         -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,
         -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,
         -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1],
       [820,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,
         -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,
         -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,
         -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  

In [12]:
# see the results
i = 0
for x in out:
    print("original_text =  ", end = '')
    for j in valid_padded_txt[i]:
        if j != 0:
            print(i2w[str(j)], end = ', ')
    print('\n')
    print("predicted text = ", end = '')
    for p in x:  
        if int(p) != -1:
            print(i2w[str(int(p))], end = ', ')    
    print('\n')
    i+=1

original_text =  note-C5_whole, note-G4_whole, note-A4_whole, note-E4_whole, note-F4_whole, note-G4_whole, note-C4_half, note-D4_half, note-E4_half, note-F4_half, note-G4_half, note-A4_half, note-B4_half, note-C5_half, 

predicted text = note-D5_eighth, 

original_text =  note-G3_whole, note-G3_half, note-G3_half, note-G3_whole, note-G3_whole, note-G3_half, note-G3_half, note-G3_half, note-G3_half, note-F#3_half., note-E3_quarter, note-D3_half., note-E3_eighth, note-F#3_eighth, note-G3_double_whole, multirest-1, 

predicted text = note-D5_eighth, 

original_text =  note-D4_double_whole, note-A4_whole, rest-half, note-A4_half, note-C5_whole, note-Bb4_half, note-A4_half, note-A4_quarter, note-G4_quarter, note-F4_half, note-E4_whole, rest-half, note-E4_half, note-F4_half., note-E4_quarter, 

predicted text = note-D5_eighth, 

original_text =  note-A4_whole, note-Bb4_whole, note-A4_whole, note-G4_whole, note-A4_whole, note-D4_whole, note-F4_whole, note-G4_whole, note-A4_whole, note-Bb4_who